In [1]:
import collections
import math
import numpy
import random

# Prepare data

In [2]:
small = [round(random.random()) for _ in range(10**5)]

In [3]:
signal = [round(random.random()) for _ in range(10**7)]

# Pure python

In [4]:
def entropy(signal: list, window: int) -> float:
    """ Computes entropy of a signal. """
    n = len(signal)//window
    
    counter = {}
    for i in range(n):
        sub = tuple(signal[i*window:(i+1)*window])
        if sub not in counter:
            counter[sub] =0
        counter[sub] += 1
        
    probs = [float(v)/n for v in counter.values()]
    n_probs = len(probs)
        
    if n_probs <= 1:
        return 0
    
    return -sum([p*math.log2(p) for p in probs])/window

In [5]:
entropy(signal, window=2)

0.9999999496525598

In [6]:
%timeit entropy(signal, window=2)

2.25 s ± 30.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit entropy(signal, window=20)

742 ms ± 4.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Interruptions

In [8]:
def entropy(signal: list, window: int) -> float:
    """ Computes entropy of a signal. """
    n = len(signal)//window
    
    counter = {}
    for i in range(n):
        sub = tuple(signal[i*window:(i+1)*window])
        try:
            counter[sub] += 1
        except KeyError:
            counter[sub] = 1
        
    probs = [float(v)/n for v in counter.values()]
    n_probs = len(probs)
        
    if n_probs <= 1:
        return 0
    return -sum([p*math.log2(p) for p in probs])/window

In [9]:
entropy(signal, window=2)

0.9999999496525598

In [10]:
%timeit entropy(signal, window=2)

1.98 s ± 5.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit entropy(signal, window=20)

659 ms ± 9.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Collections 1

In [12]:
def entropy(signal: list, window: int) -> float:
    """ Computes entropy of a signal. """
    n = len(signal)//window
    
    counter = collections.defaultdict(float)
    for i in range(n):
        sub = tuple(signal[i*window:(i+1)*window])
        counter[sub] += 1
        
    probs = [float(v)/n for v in counter.values()]
    n_probs = len(probs)
        
    if n_probs <= 1:
        return 0
    
    return -sum([p*math.log2(p) for p in probs])/window

In [13]:
entropy(signal, window=2)

0.9999999496525598

In [14]:
%timeit entropy(signal, window=2)

1.98 s ± 41.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit entropy(signal, window=20)

730 ms ± 8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Collections 2

In [16]:
def entropy(signal: list, window: int) -> float:
    """ Computes entropy of a signal. """
    n = len(signal)//window
    parts = [tuple(signal[i*window:(i+1)*window]) for i in range(n)]
    n_parts = len(parts)
    
    counts = collections.Counter(parts).values()
    probs = [v/n_parts for v in counts]
    n_probs = len(probs)

    if n_probs <= 1:
        return 0
    return -sum([p*math.log2(p) for p in probs])/window

In [17]:
entropy(signal, window=2)

0.9999999496525598

In [18]:
%timeit entropy(signal, window=2)

1.92 s ± 4.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit entropy(signal, window=20)

497 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# numpy

In [102]:
def entropy(signal: list, window: int) -> float:
    """ Computes entropy of a signal. """
    signal = [tuple(s) for s in numpy.array(signal).reshape((-1, 2))]
    n_signal = len(signal)
    
    counts = collections.Counter(signal).values()        
    probs = [v/n_signal for v in counts]
    n_probs = len(probs)
        
    if n_probs <= 1:
        return 0
    return -numpy.sum(probs*numpy.log2(probs))/window

In [103]:
entropy(signal, window=2)

0.9999999496525598

In [22]:
%timeit entropy(signal, window=2)

4.64 s ± 4.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%timeit entropy(signal, window=20)

4.66 s ± 6.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# pure numpy

In [104]:
def entropy(signal: list, window: int) -> float:
    """ Computes entropy of a signal. """
    if len(signal) < 1:
        return 0
    
    n = len(signal)//window
    signal = numpy.array(signal).reshape((-1, 2))
    counts = numpy.unique(signal, return_counts=True, axis=0)[1]
    
    probs = counts / n    
    n_probs = len(probs)
    
    return -numpy.sum(probs*numpy.log2(probs))/window

In [105]:
entropy(signal, window=2)

0.9999999496525598

In [26]:
%timeit entropy(signal, window=2)

4.12 s ± 3.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit entropy(signal, window=20)

4.21 s ± 5.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Fastest

In [139]:
def entropy(signal: list, window: int) -> float:
    """ Computes entropy of a signal. """
    n = len(signal)//window
    parts = [tuple(signal[i*window:(i+1)*window]) for i in range(n)]    
    counter = list(collections.Counter(parts).values())

    if len(counter) <= 1:
        return 0
    
    norm = math.log2(len(parts))
    return (-sum([c*math.log2(c) for c in counter])/len(parts) + norm)/window

In [140]:
entropy(signal, window=2)

0.99999994965256

In [141]:
%timeit entropy(signal, window=2)

1.93 s ± 4.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [142]:
%timeit entropy(signal, window=20)

448 ms ± 5.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
